In [15]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Importo librería de Carla

from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# Importar os para la escritura del .csv
# -----------------------------------------------------------------------
import os


# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', 1000) # Ajusta el ancho de la consola para evitar truncamientos


In [16]:
def generar_driver ():

    # Usaremos esta función para creara un único driver que reutilizaremos durante la búsqueda de datos.
    # Devuelve el driver que usaremos para hacer el web scrapping de las webs de las películas. 
    driver = webdriver.Chrome()

    sleep(2)
    driver.maximize_window()
    print("Maximizando la web.")
    sleep(2)
    driver.get("https://www.imdb.com/")
    print("Accediendo a la web.")
    sleep(2)
    try:
        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        print("Cookies aceptadas.")
        sleep(2)
    except:
        sleep(2)
    return driver

# Cerraremos este driver después de haber llamado a la función extraer_actores (). 

In [17]:

def web_scrapping (driver, id_pelicula):

    # Esta función recibe un driver y un identificador de película, lo que le permite hacer web scrapping.
    # Devuelve una lista con los datos de los 10 intérpretes de la película.

    print("Construyendo url con el identificador")

    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"     # Construimos url con el identificador pasado por parámetro.

    print(f"Entrando en la web {url_pelicula}")

    driver.get(url_pelicula)

    sleep(2)

    try:
        nombre_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-1f50b7c-0.iPPbjm > h1 > span").text
        print(f"Entrando en la web de la película {nombre_pelicula}: {url_pelicula}")
        sleep(2)
    except:
        print("No se ha podido leer el nombre de la película")

    # Inicializo el diccionario donde guardaré las URL y el nombre de cada intérprete.

    actores_url = {} 
    lista_actores = []

    # De la página de la película extraigo los nombres y urls de los 10 actores principales.

    try:
        elemento_casting = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
        print("Explorando el casting")
        sleep(2)

    except:
        print("No hemos podido acceder al casting de la película.")
        # return
    
    print("Extraigo el nombre y url de los actores")  
    for i in range (1, 11):
        try:
            actor = elemento_casting.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.kSFMrr > a")
            nombre_actor = actor.text
            # nombre.append(nombre_actor)
            actores_url[nombre_actor] = actor.get_attribute("href")
        except:
            print("No se ha encontrado o no se ha podido extraer la url de algún actor")
    print("Extraigo el nombre y url de los actores")
    sleep(2)

    # Ahora que tengo sus direcciones url, exploro cada web en busca de los datos restantes del actor.

    for actor, url in actores_url.items():
        print(f"Entrando en la url de {actor}")
        driver.get(url)
        sleep(2)
        print("\tExtraemos año de nacimiento")

        try: 
            fecha_completa = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-11.hFWIYv > section > aside > div > span:nth-child(2)").text
            print(f"fecha_completa es {fecha_completa}")
            lista_year = fecha_completa.split()
            # print(f"lista_year es {lista_year}")
            year = int(lista_year[-1])
            # print(f"year es {year}")

            # Nos aseguramos de que devuelva un valor numérico o no devuelva nada.

        except: 
            print("No se ha podido extraer el año de nacimiento.")
            year = -1    # En caso de no haber podido extraer, devolverá el año -1, que claramente nos indicará que es un error.

        print(f"\tExtraemos otros roles en la industria")
    
        sleep(2)
        lista_roles = []
        roles = ""
        try:
            elemento_roles= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div > ul").find_elements("css selector", "li")
            for rol in elemento_roles:
                lista_roles.append(rol.text)
            roles = ". ".join(lista_roles)
        except:
            print("Ha ocurrido un error con los roles del intérprete.")
            print(f"Roles de interprete es un tipo {type(roles)}. Contiene ahora mismo:{roles} ")

                
        sleep(2)

        print("\tRecopilando otras películas por las que se le conoce")


        try:
                selector_meta = driver.find_element("css selector", "head > meta:nth-child(13)")
                conocido_por = selector_meta.get_attribute("content")
                print(conocido_por)
                # Limpiamos el contenido
                conocido_por = conocido_por.replace("Conocido/a por: ", "")
            
        except: 
                print("No se han podido extraer otras películas del intérprete")
                conocido_por = ""

        

        print("\tRecopilando premios")
        # Al estar en la versión MPV, de momento nos vamos a quedar solo con el resumen de los premios. 
        resumen_premios = ""

        try: 
            resumen_premios = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
            sleep(2)
            if resumen_premios == "Premios":
                resumen_premios = "Otros premios"

        except:
            print(f"{actor} no tiene resumen de premios")
            resumen_premios = "No tiene premios"
        

        # Creamos una tupla con los datos del actor recién explorado y la añadimos a la lista.
        
        tupla_actor = (actor, year, roles, conocido_por, resumen_premios)
        print(tupla_actor)
        lista_actores.append(tupla_actor)       
    
    print("Finalizamos exploración de los actores principales")
    return lista_actores



In [18]:

def crear_csv (nombre = "data_actores"):

    # Esta función crea un archivo CSV vacío con el nombre que se le pasa por parámetro (por defecto: data_actores)
    # El archivo se irá escribiendo a medida que exploramos los actores.
    # Devuelve el nombre del archivo.
    
    data = {
        'Nombre': [],
        'Anyo': [],
        'Roles': [],
        'Peliculas': [],
        'Premios': []
    }

    df = pd.DataFrame(data)
    nombre_archivo = f'{nombre}.csv'
    df.to_csv(f'{nombre}.csv', index=False)  
    print(f"Archivo {nombre_archivo} (vacío) creado correctamente")
    return nombre_archivo

    # Guardamos los nombres de las columnas en un csv que ahora está vacío de datos.



A partir del fichero de películas de la API, que recoge 15 años de datos (desde 1999 a 2009), extraemos una lista de identificadores de cada película. 

In [19]:
def extractor_idowners (archivo_csv) :

    # Esta función recibe un archivo .csv y devuelve una lista con todos los idowners que hay en el archivo.

    df_pelis_API = pd.read_csv(archivo_csv) 
    lista_idowners = df_pelis_API['IdOwner'].tolist()
    return lista_idowners

In [20]:
identificadores_1999_2009 = extractor_idowners("peliculas_api_desde_1995_a_2009.csv")

In [21]:
mi_driver = generar_driver()

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.


In [22]:
print(identificadores_1999_2009)

['tt0100267', 'tt0102739', 'tt0105096', 'tt0105298', 'tt0106616', 'tt0107070', 'tt0102952', 'tt0101616', 'tt0102532', 'tt0108576', 'tt0107537', 'tt0105881', 'tt0109178', 'tt0109033', 'tt0109536', 'tt0109400', 'tt0109746', 'tt0109905', 'tt0110252', 'tt0110826', 'tt0111359', 'tt0109546', 'tt0109415', 'tt0109723', 'tt0110604', 'tt0111574', 'tt0111464', 'tt0111244', 'tt0109028', 'tt0109642', 'tt0110217', 'tt0111151', 'tt0111269', 'tt0109363', 'tt0109489', 'tt0095102', 'tt0109034', 'tt0109532', 'tt0099840', 'tt0103655', 'tt0106322', 'tt0106621', 'tt0106421', 'tt0107025', 'tt0109239', 'tt0110028', 'tt0110294', 'tt0110910', 'tt0110463', 'tt0111678', 'tt0111129', 'tt0112526', 'tt0112803', 'tt0113035', 'tt0112946', 'tt0112923', 'tt0112908', 'tt0113981', 'tt0114485', 'tt0111305', 'tt0112632', 'tt0113643', 'tt0113591', 'tt0113682', 'tt0114170', 'tt0111645', 'tt0113443', 'tt0113314', 'tt0112675', 'tt0113033', 'tt0113489', 'tt0109827', 'tt0110125', 'tt0102267', 'tt0108233', 'tt0109136', 'tt0110331'

In [25]:


def extraer_datos_actores (lista_id, nombre_fichero = "data_actores"):

    # Esta función recibe una lista con identificadores de películas y un string con el nombre del fichero.csv 
    # donde guardaremos los datos de los 10 actores principales de cada película.

    # La función también devuelve un dataframe con los datos recopilados.
    
    mi_driver = generar_driver() # Creamos el driver que reutilizaremos para explorar la web Imdb.

    nombre_archivo_completo = crear_csv(nombre_fichero) # Creamos eL archivo vacío con el nombre de las columnas de los datos de los actores.

    columnas = ["Nombre", "Anyo", "Roles", "Peliculas", "Premios"]

    for id in lista_id: 
        data_actores = web_scrapping (mi_driver, id)
        df1 = pd.DataFrame(data_actores, columns=columnas)
        path = nombre_archivo_completo
        df1.to_csv(path, index=None, mode="a", header=not os.path.isfile(path)) # Añadimos todos los datos extraídos de la exploración de la última película explorada.
        print(f"Datos de película con {id} añadidos al fichero")
    
    mi_driver.close()
    print("Cerrando el navegador. Exploración terminada. Ya puede consultar su archivo {nombre_archivo_completo}")
    df = pd.read_csv(nombre_archivo_completo)
    return df


# DOCU: https://es.stackoverflow.com/questions/278871/a%C3%B1adir-datos-a-csv-sin-sobreescribir-el-contenido-previo







Guardar el dataframe que devuelve la funcion anterior.

Hasta aquí, el código. A partir de este momento, hacemos pruebas y extracciones.

ULTIMA EXTRACCIÓN

In [ ]:
identificadores_2004_2005 = extractor_idowners("peliculas_desde_tt0138749.csv")
identificadores_2004_2005 = extractor_idowners("peliculas_desde_tt0138749.csv")
identificadores_2004_2005 = extractor_idowners("peliculas_desde_tt0138749.csv")
identificadores_2004_2005 = extractor_idowners("peliculas_desde_tt0138749.csv")


In [ ]:
df_actores = extraer_datos_actores(identificadores_2001_2004 , "data_actores_desde_2001")

In [28]:
identificadores_2001_2004 = extractor_idowners("peliculas_desde_tt0138749.csv")


In [29]:
df_actores = extraer_datos_actores(identificadores_2001_2004 , "data_actores_desde_2001")

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Archivo data_actores_desde_2001.csv (vacío) creado correctamente
Construyendo url con el identificador
Entrando en la web https://www.imdb.com/title/tt0138749/
No se ha podido leer el nombre de la película
Explorando el casting
Extraigo el nombre y url de los actores
Extraigo el nombre y url de los actores
Entrando en la url de Kevin Kline
	Extraemos año de nacimiento
fecha_completa es 24 de octubre de 1947
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
Conocido/a por: Wild Wild West, Dave, presidente por un día, Un pez llamado Wanda
	Recopilando premios
('Kevin Kline', 1947, 'Reparto. Dirección. Producción', 'Wild Wild West, Dave, presidente por un día, Un pez llamado Wanda', 'Ganó 1 premio Óscar')
Entrando en la url de Kenneth Branagh
	Extraemos año de nacimiento
fecha_completa es 10 de diciembre de 1960
	Extraemos otros roles en la industria
	Recopilando otras películas por las 

In [29]:
lista_ids_pelis_2000 = ['tt0102362', 'tt0113092', 'tt0113086', 'tt0116748', 'tt0116391', 'tt0115937', 'tt0118694', 'tt0119495', 'tt0119806', 'tt0120202', 'tt0123003', 'tt0120263', 'tt0123055', 'tt0120630', 'tt0116628', 'tt0119273', 'tt0120755', 'tt0120913', 'tt0120673', 'tt0120903', 'tt0120917', 'tt0130623', 'tt0132910', 'tt0131704', 'tt0138749', 'tt0144964', 'tt0144574', 'tt0146984', 'tt0128158', 'tt0134084', 'tt0134983', 'tt0134847', 'tt0137115', 'tt0144084', 'tt0148120', 'tt0156020', 'tt0120753', 'tt0127349', 'tt0160495', 'tt0160644', 'tt0160289', 'tt0161081', 'tt0162711', 'tt0113026', 'tt0119866', 'tt0122459', 'tt0119966', 'tt0120467', 'tt0122161', 'tt0120117', 'tt0120698']


ALMACEN PARA BORRAR

In [30]:

# Este código ha sido superado por una opción mejor. Lo guardamos de momento pero ya no sirve.

def extraer_datos_actores (lista_ids):

    # Recibe una lista de identificadores de películas.
    # Devuelve un dataframe con los datos de los actores de esas pelicula.

    mi_driver = generar_driver()
    datos_actores_total = []

    for id in lista_ids:
        datos_actores = web_scrapping(mi_driver, id)     # Llamamos a nuestra función para sacar datos de esa película
        datos_actores_total.extend(datos_actores)        # Añadimos el resultado a una lista que contendrá todos los datos de la lista de identificadores.

    columnas = ["Nombre Intérprete", "Año de nacimiento", "Roles en la industria", "Conocido por", "Premios"]
    df_actores = pd.DataFrame(datos_actores_total, columns=columnas)
    mi_driver.close() 
    print("Cerrando navegador")
    print(df_actores)
    return df_actores